In [1]:
from guppylang.decorator import guppy

ModuleNotFoundError: No module named 'guppylang'

Test that functions registered inside calls still produce correct diagnostic spans. See https://github.com/CQCL/guppylang/issues/906

In [ ]:
def make():    
    @guppy
    def foo() -> int:    
        return 1.0

    return foo

foo = make()

@guppy
def main() -> int:
    return foo()

main.compile();

Error: Type mismatch (at <In[2]>:4:15)
  | 
2 |     @guppy
3 |     def foo() -> int:
4 |         return 1.0
  |                ^^^ Expected return value of type `int`, got `float`

Guppy compilation failed due to 1 previous error
